In [1]:
import os
import sys
folder = os.path.dirname('../../../algorithms/')
sys.path.append(folder)

from PG.trainer import PG_Trainer

In [2]:
env_name = 'CartPole-v1'
lr_base = 0.0001
lr_decay = 0.0001

random_seed = 42
gamma = 0.99                # discount for future rewards
batch_size = 64         # num of transitions sampled from replay buffer

max_episodes = 100000         # max num of episodes
max_timesteps = 3000        # max timesteps in one episode
log_interval = 50           # print avg reward after interval

entropy_beta = 0.01
bellman_steps = 50
baseline_steps = 50000


In [3]:
config = [
     {'dim': [None, 128], 'dropout': False, 'activation': 'relu'},    
     {'dim': [128, None], 'dropout': False, 'activation': False},    
] 

In [4]:
agent = PG_Trainer(env_name, config, random_seed=random_seed, lr_base=lr_base, lr_decay=lr_decay, 
                   gamma=gamma, batch_size=batch_size,
                   max_episodes=max_episodes, max_timesteps=max_timesteps, 
                   log_interval=log_interval, entropy_beta=entropy_beta, 
                   bellman_steps=bellman_steps, baseline_steps=baseline_steps
                   )
agent.train()

NETWORK: Sequential(
  (0): Linear(in_features=4, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=2, bias=True)
) Device: cuda:0
Random Seed: 42
action_space=Discrete(2)
obs_space=Box(4,)
threshold=475.0 

DIR=./preTrained/pg NAME=pg_CartPole-v1_42
No models to load

Training started ... 
Ep:   50  Rew:   28.00  Avg Rew:   23.10  LR:0.00009951  Loss: -1.845298
Ep:  100  Rew:   16.00  Avg Rew:   25.23  LR:0.00009902  Loss: 0.007866
Ep:  150  Rew:   26.00  Avg Rew:   26.17  LR:0.00009853  Loss: -0.247397
Ep:  200  Rew:   30.00  Avg Rew:   26.59  LR:0.00009805  Loss: 0.039433
Ep:  250  Rew:   13.00  Avg Rew:   27.50  LR:0.00009757  Loss: 0.088303
Ep:  300  Rew:   45.00  Avg Rew:   26.81  LR:0.00009710  Loss: 0.339025
Ep:  350  Rew:   25.00  Avg Rew:   29.58  LR:0.00009663  Loss: 0.321097
Ep:  400  Rew:   25.00  Avg Rew:   30.35  LR:0.00009616  Loss: 0.579626
Ep:  450  Rew:   26.00  Avg Rew:   27.71  LR:0.00009570  Loss: 0.331722
Ep:  500  Rew:  100.0

Ep: 5500  Rew:  463.00  Avg Rew:  447.87  LR:0.00006452  Loss: -0.043458
Ep: 5550  Rew:  345.00  Avg Rew:  445.01  LR:0.00006431  Loss: 0.031269
Ep: 5600  Rew:  209.00  Avg Rew:  456.00  LR:0.00006411  Loss: 0.044635
Ep: 5650  Rew:  294.00  Avg Rew:  461.13  LR:0.00006390  Loss: -0.004703
Ep: 5700  Rew:  424.00  Avg Rew:  431.53  LR:0.00006370  Loss: -0.170769
Ep: 5750  Rew:  466.00  Avg Rew:  407.45  LR:0.00006350  Loss: 0.064876
Ep: 5800  Rew:  500.00  Avg Rew:  406.40  LR:0.00006330  Loss: 0.013180
Ep: 5850  Rew:  500.00  Avg Rew:  415.01  LR:0.00006310  Loss: 0.161146
Ep: 5900  Rew:  500.00  Avg Rew:  448.79  LR:0.00006290  Loss: 0.089693
Ep: 5950  Rew:  500.00  Avg Rew:  465.48  LR:0.00006270  Loss: -0.035755
Ep: 6000  Rew:  291.00  Avg Rew:  431.91  LR:0.00006250  Loss: -0.065503
Ep: 6050  Rew:  335.00  Avg Rew:  410.97  LR:0.00006231  Loss: -0.069374
Ep: 6100  Rew:  420.00  Avg Rew:  429.89  LR:0.00006212  Loss: 0.020865
Ep: 6150  Rew:  194.00  Avg Rew:  439.65  LR:0.00006192  L

In [5]:
agent.test()

Test episode: 1	Reward: 500.00
Test episode: 2	Reward: 267.00
Test episode: 3	Reward: 500.00
